## Convert videos to images

In [1]:
import glob
import os
import cv2

folders = ['Acidovorax/GS80PL20Z20/','Acidovorax/GS40PL20Z20/','Geobacter/GS80PL20Z20/','Geobacter/GS40PL10Z20/','Acidovorax/GS40PL10Z20/','Paenibacillus/GS80PL20Z20/' ]

for folder in folders:
    os.chdir('E:/Flow/')
    files=glob.glob(folder+'*1x1y_5min*.avi')
    os.chdir(folder) 
    for file in files:
        print ('Saving images for ' + str(file))
        partial=0
        vidcap=cv2.VideoCapture(file[23:])
        try:
            os.makedirs(file[23:-4]+'_frames')
        except:
            if len(os.listdir(file[23:-4]+'_frames'))>0:
                partial = 1
        os.chdir(file[23:-4]+'_frames')
        success,image = vidcap.read()
        if partial == 0:
            count = 0
        else:
            count = len(os.listdir())  
        while success:
            cv2.imwrite("frame%d.jpg" % count, image)     # save frame as JPEG file      
            success,image = vidcap.read()
            #print('Read a new frame: ', success)
            count += 1
        os.chdir(os.path.dirname(os.getcwd()))


Saving images for Acidovorax/GS80PL20Z20\FR_1ulh_20X_25fps_1x1y_5min_001.avi
Saving images for Acidovorax/GS80PL20Z20\FR_1ulh_20X_25fps_1x1y_5min_002.avi
Saving images for Acidovorax/GS80PL20Z20\FR_1ulh_20X_25fps_1x1y_5min_004.avi
Saving images for Acidovorax/GS80PL20Z20\FR_1ulh_20X_25fps_1x1y_5min_005.avi
Saving images for Acidovorax/GS80PL20Z20\FR_5ulh_20X_25fps_1x1y_5min_001.avi
Saving images for Acidovorax/GS80PL20Z20\FR_5ulh_20X_25fps_1x1y_5min_002.avi
Saving images for Acidovorax/GS80PL20Z20\FR_5ulh_20X_25fps_1x1y_5min_003.avi
Saving images for Acidovorax/GS40PL20Z20\FR_1ulh_20X_25fps_1x1y_5min_001.avi
Saving images for Acidovorax/GS40PL20Z20\FR_1ulh_20X_25fps_1x1y_5min_002.avi
Saving images for Acidovorax/GS40PL20Z20\FR_1ulh_20X_25fps_1x1y_5min_003.avi
Saving images for Acidovorax/GS40PL20Z20\FR_5ulh_20X_25fps_1x1y_5min_001.avi
Saving images for Acidovorax/GS40PL20Z20\FR_5ulh_20X_25fps_1x1y_5min_002.avi
Saving images for Geobacter/GS80PL20Z20\FR_1ulh_20X_25fps_1x1y_5min_001.avi


## Particle Tracking with Trackmate

In [ ]:
import sys

from ij import IJ
from ij import WindowManager

from fiji.plugin.trackmate import Model
from fiji.plugin.trackmate import Settings
from fiji.plugin.trackmate import TrackMate
from fiji.plugin.trackmate import SelectionModel
from fiji.plugin.trackmate import Logger
from fiji.plugin.trackmate.detection import LogDetectorFactory
from fiji.plugin.trackmate.tracking import LAPUtils
from fiji.plugin.trackmate.tracking.sparselap import SparseLAPTrackerFactory
from fiji.plugin.trackmate.gui.displaysettings import DisplaySettingsIO
import fiji.plugin.trackmate.visualization.hyperstack.HyperStackDisplayer as HyperStackDisplayer
import fiji.plugin.trackmate.features.FeatureFilter as FeatureFilter

# We have to do the following to avoid errors with UTF8 chars generated in 
# TrackMate that will mess with our Fiji Jython.
reload(sys)
sys.setdefaultencoding('utf-8')

# Get currently selected image
# imp = WindowManager.getCurrentImage()
imp = IJ.openImage('https://fiji.sc/samples/FakeTracks.tif')
imp.show()


#----------------------------
# Create the model object now
#----------------------------

# Some of the parameters we configure below need to have
# a reference to the model at creation. So we create an
# empty model now.

model = Model()

# Send all messages to ImageJ log window.
model.setLogger(Logger.IJ_LOGGER)



#------------------------
# Prepare settings object
#------------------------

settings = Settings(imp)

# Configure detector - We use the Strings for the keys
settings.detectorFactory = LogDetectorFactory()
settings.detectorSettings = {
    'DO_SUBPIXEL_LOCALIZATION' : True,
    'RADIUS' : 2.5,
    'TARGET_CHANNEL' : 1,
    'THRESHOLD' : 0.,
    'DO_MEDIAN_FILTERING' : False,
}  

# Configure spot filters - Classical filter on quality
filter1 = FeatureFilter('QUALITY', 30, True)
settings.addSpotFilter(filter1)

# Configure tracker - We want to allow merges and fusions
settings.trackerFactory = SparseLAPTrackerFactory()
settings.trackerSettings = LAPUtils.getDefaultLAPSettingsMap() # almost good enough
settings.trackerSettings['ALLOW_TRACK_SPLITTING'] = True
settings.trackerSettings['ALLOW_TRACK_MERGING'] = True

# Add ALL the feature analyzers known to TrackMate. They will 
# yield numerical features for the results, such as speed, mean intensity etc.
settings.addAllAnalyzers()

# Configure track filters - We want to get rid of the two immobile spots at
# the bottom right of the image. Track displacement must be above 10 pixels.

filter2 = FeatureFilter('TRACK_DISPLACEMENT', 10, True)
settings.addTrackFilter(filter2)


#-------------------
# Instantiate plugin
#-------------------

trackmate = TrackMate(model, settings)

#--------
# Process
#--------

ok = trackmate.checkInput()
if not ok:
    sys.exit(str(trackmate.getErrorMessage()))

ok = trackmate.process()
if not ok:
    sys.exit(str(trackmate.getErrorMessage()))


#----------------
# Display results
#----------------

# A selection.
selectionModel = SelectionModel( model )

# Read the default display settings.
ds = DisplaySettingsIO.readUserDefault()

displayer =  HyperStackDisplayer( model, selectionModel, imp, ds )
displayer.render()
displayer.refresh()

# Echo results with the logger we set at start:
model.getLogger().log( str( model ) )
